# Use CASH_OUT_Template Customized Notebook Template

In [36]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

import pandas as pd
import datetime as dt

In [37]:
#import seaborn as sns
import subprocess

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

In [38]:
from fosforml import register_model
import requests

In [39]:
# Import label encoder 
from sklearn import preprocessing 
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from math import sqrt
import numpy as np

In [40]:
table_name = 'ATM_TRANSACTION_MASTER_DATA'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [41]:
table_name = 'ATM_TRANSACTION_MASTER_DATA'

sf_df = my_session.sql("select * from {}".format(table_name))
df_original = sf_df.to_pandas()

In [42]:
df.columns

Index(['DATE', 'WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'BANK_NAME',
       'ATM_ID', 'DISPENSED_AMOUNT', 'TOTAL_TRANSACTION_COUNT',
       'DOWNTIME_IN_MINS', 'ATM_MAX_CAPACITY', 'MALE_POPULATION',
       'FEMALE_POPULATION', 'TOTAL_POPULATION', 'NO_OF_VETERANS',
       'FOREIGN_BORN', 'AVG_HOUSEHOLD_SIZE', 'LANDAREA_SQ_MILE',
       'POPULATION_RATIO', 'POPULATION_DENSITY', 'MEDIAN_HOUSE_INCOME',
       'LOCATION_TYPE', 'ATM_PLACEMENTS', 'ATM_ACCESSIBILITY',
       'ATM_MAINTAINENANCE', 'ATM__REPLENISHMENT'],
      dtype='object')

In [43]:
#to_drop = ['ATM_STATE_DETAILS_STATE','MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
to_drop = ['MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
df.drop(to_drop, inplace=True, axis=1)

In [44]:
#to_drop = ['ATM_STATE_DETAILS_STATE','MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
to_drop = ['MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
df_original.drop(to_drop, inplace=True, axis=1)

In [45]:
df.shape

(312417, 16)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  object 
 2   HOLIDAY_FLAG             312417 non-null  object 
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
 11  LOCATION_TYPE            312417 non-null  object 
 12  ATM_PLACEMENTS           312417 non-null  object 
 13  ATM_ACCESSIBILITY        312417 non-null  object 
 14  ATM_

In [47]:
df.head()

,DATE,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,BANK_NAME,ATM_ID,DISPENSED_AMOUNT,TOTAL_TRANSACTION_COUNT,DOWNTIME_IN_MINS,ATM_MAX_CAPACITY,LOCATION_TYPE,ATM_PLACEMENTS,ATM_ACCESSIBILITY,ATM_MAINTAINENANCE,ATM__REPLENISHMENT
0,2023-01-29,Y,N,3,Maine,Citibank,TBH000274025,225180.0,90.0,0.0,2860000.0,Urban,Drive-Thru,Restrcited hours,Irregular,Weekly
1,2023-01-30,Y,N,3,Maine,Citibank,TBH000274025,262440.0,117.0,0.0,2860000.0,Urban,Drive-Thru,Restrcited hours,Irregular,Weekly
2,2023-01-31,Y,N,3,Maine,Citibank,TBH000274025,561150.0,160.0,0.0,2860000.0,Urban,Drive-Thru,Restrcited hours,Irregular,Weekly
3,2023-02-01,Y,N,3,Maine,Citibank,TBH000274025,437220.0,129.0,0.0,2860000.0,Urban,Drive-Thru,Restrcited hours,Irregular,Weekly
4,2023-02-02,Y,N,3,Maine,Citibank,TBH000274025,360900.0,119.0,0.0,2860000.0,Urban,Drive-Thru,Restrcited hours,Irregular,Weekly


In [48]:
df.columns

Index(['DATE', 'WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'BANK_NAME',
       'ATM_ID', 'DISPENSED_AMOUNT', 'TOTAL_TRANSACTION_COUNT',
       'DOWNTIME_IN_MINS', 'ATM_MAX_CAPACITY', 'LOCATION_TYPE',
       'ATM_PLACEMENTS', 'ATM_ACCESSIBILITY', 'ATM_MAINTAINENANCE',
       'ATM__REPLENISHMENT'],
      dtype='object')

In [49]:
df.isnull().sum()

DATE                       0
WEEKDAY_FLAG               0
HOLIDAY_FLAG               0
BANK_ID                    0
STATE                      0
BANK_NAME                  0
ATM_ID                     0
DISPENSED_AMOUNT           0
TOTAL_TRANSACTION_COUNT    0
DOWNTIME_IN_MINS           0
ATM_MAX_CAPACITY           0
LOCATION_TYPE              0
ATM_PLACEMENTS             0
ATM_ACCESSIBILITY          0
ATM_MAINTAINENANCE         0
ATM__REPLENISHMENT         0
dtype: int64

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  object 
 2   HOLIDAY_FLAG             312417 non-null  object 
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
 11  LOCATION_TYPE            312417 non-null  object 
 12  ATM_PLACEMENTS           312417 non-null  object 
 13  ATM_ACCESSIBILITY        312417 non-null  object 
 14  ATM_

In [51]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['HOUR'] = df['DATE'].dt.hour
df['DAYOFWEEK'] = df['DATE'].dt.dayofweek
df['QUARTER'] = df['DATE'].dt.quarter
df['MONTH'] = df['DATE'].dt.month
df['YEAR'] = df['DATE'].dt.year
df['DAYOFYEAR'] = df['DATE'].dt.dayofyear
df['DAYOFMONTH'] = df['DATE'].dt.day
df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")

In [52]:
df['WEEKDAY_FLAG'] = df['WEEKDAY_FLAG'].map({'Y': 1, 'N': 0})
df['HOLIDAY_FLAG'] = df['HOLIDAY_FLAG'].map({'Y': 1, 'N': 0})

In [53]:
df.head()

,DATE,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,BANK_NAME,ATM_ID,DISPENSED_AMOUNT,TOTAL_TRANSACTION_COUNT,DOWNTIME_IN_MINS,...,ATM_ACCESSIBILITY,ATM_MAINTAINENANCE,ATM__REPLENISHMENT,HOUR,DAYOFWEEK,QUARTER,MONTH,YEAR,DAYOFYEAR,DAYOFMONTH
0,2023-01-29 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,225180.0,90.0,0.0,...,Restrcited hours,Irregular,Weekly,0,6,1,1,2023,29,29
1,2023-01-30 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,262440.0,117.0,0.0,...,Restrcited hours,Irregular,Weekly,0,0,1,1,2023,30,30
2,2023-01-31 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,561150.0,160.0,0.0,...,Restrcited hours,Irregular,Weekly,0,1,1,1,2023,31,31
3,2023-02-01 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,437220.0,129.0,0.0,...,Restrcited hours,Irregular,Weekly,0,2,1,2,2023,32,1
4,2023-02-02 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,360900.0,119.0,0.0,...,Restrcited hours,Irregular,Weekly,0,3,1,2,2023,33,2


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  int64  
 2   HOLIDAY_FLAG             312417 non-null  int64  
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
 11  LOCATION_TYPE            312417 non-null  object 
 12  ATM_PLACEMENTS           312417 non-null  object 
 13  ATM_ACCESSIBILITY        312417 non-null  object 
 14  ATM_

In [55]:
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['STATE']= label_encoder.fit_transform(df['STATE'])

In [56]:
# how to understand word labels. 
atm_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['ATM_ID']= atm_encoder.fit_transform(df['ATM_ID'])

In [57]:
df.drop(['DATE','BANK_NAME','TOTAL_TRANSACTION_COUNT','DOWNTIME_IN_MINS','ATM_MAX_CAPACITY'], axis=1,inplace=True)

In [58]:
df.columns

Index(['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID',
       'DISPENSED_AMOUNT', 'LOCATION_TYPE', 'ATM_PLACEMENTS',
       'ATM_ACCESSIBILITY', 'ATM_MAINTAINENANCE', 'ATM__REPLENISHMENT', 'HOUR',
       'DAYOFWEEK', 'QUARTER', 'MONTH', 'YEAR', 'DAYOFYEAR', 'DAYOFMONTH'],
      dtype='object')

In [59]:
features = ['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID',
    'HOUR', 'DAYOFWEEK', 'QUARTER', 'MONTH', 'YEAR',
       'DAYOFYEAR', 'DAYOFMONTH']

In [60]:
print (df.shape)
df = df[df['DISPENSED_AMOUNT'] > 0]
print (df.shape)

(312417, 18)
(257564, 18)


In [61]:
X = df[features]
y = df['DISPENSED_AMOUNT']

In [62]:
X.head()

,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,ATM_ID,HOUR,DAYOFWEEK,QUARTER,MONTH,YEAR,DAYOFYEAR,DAYOFMONTH
0,1,0,3,18,299,0,6,1,1,2023,29,29
1,1,0,3,18,299,0,0,1,1,2023,30,30
2,1,0,3,18,299,0,1,1,1,2023,31,31
3,1,0,3,18,299,0,2,1,2,2023,32,1
4,1,0,3,18,299,0,3,1,2,2023,33,2


In [63]:
y

0         225180.0
1         262440.0
2         561150.0
3         437220.0
4         360900.0
            ...   
312412    195300.0
312413    245160.0
312414    222480.0
312415    198360.0
312416    314190.0
Name: DISPENSED_AMOUNT, Length: 257564, dtype: float64

In [64]:
# Split the data into training and test sets. (0.75, 0.25) split.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

In [65]:
print(f'Total # of sample in whole dataset: {len(X)}')
print("*****"*10)
print(f'Total # of sample in train dataset: {len(X_train)}')
print(f'Shape of X_train: {X_train.shape}')
print("*****"*10)
print(f'Total # of sample in test dataset: {len(X_test)}')
print(f'Shape of X_test: {X_test.shape}')

Total # of sample in whole dataset: 257564
**************************************************
Total # of sample in train dataset: 193173
Shape of X_train: (193173, 12)
**************************************************
Total # of sample in test dataset: 64391
Shape of X_test: (64391, 12)


# Linear Regression

In [66]:
Lreg = LinearRegression()

In [67]:
Lreg.fit(X_train,  y_train)

LinearRegression()

In [68]:
predictions = Lreg.predict(X_test)

In [69]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 194804.07995551525
Mean Squared Error: 61359397193.44236
Root Mean Squared Error: 247708.29052222366
r2_score: 0.1159294149693143


# Lasso Regression

In [70]:
Lreg = Lasso()

In [71]:
Lreg.fit(X_train,  y_train)
predictions = Lreg.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 194834.66575604852
Mean Squared Error: 61406024201.596275
Root Mean Squared Error: 247802.3894186581
r2_score: 0.11525760969966814


# Ridge Regression

In [72]:
Lreg = Ridge()

In [73]:
Lreg.fit(X_train,  y_train)
predictions = Lreg.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 194803.7912526467
Mean Squared Error: 61359462762.85284
Root Mean Squared Error: 247708.42287425924
r2_score: 0.11592847024055664


# K-Nearest Neighbour

In [74]:
Lreg = KNeighborsRegressor()

In [75]:
Lreg.fit(X_train,  y_train)

KNeighborsRegressor()

In [76]:
predictions = Lreg.predict(X_test)

In [77]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 132988.27196347315
Mean Squared Error: 33767038749.341366
Root Mean Squared Error: 183758.09845920088
r2_score: 0.5134820896663818


# Hyperparameter tunning

In [78]:
for i in range(2,20,2):
    #Train Model and Predict  
    Lreg = KNeighborsRegressor(n_neighbors = i).fit(X_train,y_train)
    predictions = Lreg.predict(X_test)
    print ("--------------- Neighbour: ", i,"------------------------")
    print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
    print('Mean Squared Error:', mean_squared_error(y_test,predictions))
    print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
    print('r2_score:', r2_score(y_test,predictions))

--------------- Neighbour:  2 ------------------------
Mean Absolute Error: 148112.4518178006
Mean Squared Error: 43178332541.14511
Root Mean Squared Error: 207793.9665657911
r2_score: 0.3778834953355886
--------------- Neighbour:  4 ------------------------
Mean Absolute Error: 134264.1737199298
Mean Squared Error: 34748861342.28328
Root Mean Squared Error: 186410.46468018708
r2_score: 0.4993359194978251
--------------- Neighbour:  6 ------------------------
Mean Absolute Error: 132096.04160519328
Mean Squared Error: 33210915939.588997
Root Mean Squared Error: 182238.62362185738
r2_score: 0.5214947468999054
--------------- Neighbour:  8 ------------------------
Mean Absolute Error: 131684.03878259385
Mean Squared Error: 32838775141.698643
Root Mean Squared Error: 181214.7210954415
r2_score: 0.5268565781426001
--------------- Neighbour:  10 ------------------------
Mean Absolute Error: 131687.01950583156
Mean Squared Error: 32781727008.07226
Root Mean Squared Error: 181057.24787500847


In [79]:
tree = KNeighborsRegressor(n_neighbors=10)

tree.fit(X_train,  y_train)
predictions = tree.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 131687.01950583156
Mean Squared Error: 32781727008.07226
Root Mean Squared Error: 181057.24787500847
r2_score: 0.5276785317336854


In [80]:
y_pred = predictions
y_prediction = pd.Series(y_pred)

In [50]:
type(X_train), type(X_test), type(y_train), type(y_test), type(y_pred), type(y_prediction)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.series.Series,
 numpy.ndarray,
 pandas.core.series.Series)

In [51]:
## registering the model in Fosfor.
model_reg = register_model(tree,
               score, 
               name="ATM_DispenseAMT_KNN_10_Regression", 
               description="ATM Dispense Amount KNN_10 Regression",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               #init_script="pip install snowflake-ml-python==1.0.11",
               init_script="pip install cloudpickle==2.2.1",
               y_true=y_test,
               y_pred=y_prediction,
               #prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
